In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import AdamW, ViTImageProcessor, ViTForImageClassification
from NWRD_dataset import NWRD
from tqdm import tqdm
import numpy as np
import torch.nn.functional as F
import os
from PIL import Image
from torchvision.utils import save_image
from torchvision import transforms


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
CUDA_LAUNCH_BLOCKING=1
TORCH_USE_CUDA_DSA=1
print(device)

cuda


In [ ]:
transformations = transforms.Compose([
    transforms.ToTensor()            # Convert the image to a PyTorch tensor
])

In [ ]:
#test_ds = NWRD(root_dir="/media/tukl/2dceb8df-2a02-4725-92fb-525b406c4482/Hirra/DATASETS/NWRD-classifier/test", train=False, transform=transformations)
#test_ds = NWRD(root_dir="/home/Hirra/DATASETS/NWRD-classifier/test", train=False, transform=transformations)
test_ds = NWRD(root_dir="/home/Hirra/DATASETS/NWRD_FINAL_Filtered_Masks_threshold9_classifier", train=False, transform=transformations)


test_loader = DataLoader(test_ds, batch_size=1, shuffle=False)


In [ ]:
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224')
model = ViTForImageClassification.from_pretrained('google/vit-base-patch16-224')
model.classifier = torch.nn.Linear(768,2)
model.to(device)

ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=7

In [ ]:

#model_weights = torch.load('/home/Hirra/coding_files/crossvit/weights/wandb_vit_base_final_hard_val_NWRD_epoch_50_lr_0.000000001_wd_0.001_batch_size_8_unaugmented_unequlaized/49.pth') #use 33.pth the lowest
model_weights = torch.load('/home/Hirra/coding_files/crossvit/weights/wandb_vit_base_final_val_NWRD_epoch_50_lr_0.000000003_wd_0.001_batch_size_8_unaugmented_unequlaized/28.pth') #use 33.pth the lowest
model.load_state_dict(model_weights.state_dict())

<All keys matched successfully>

In [ ]:
criterion = torch.nn.CrossEntropyLoss()

In [ ]:
#Testing

true_positives = 0
false_positives = 0
true_negatives = 0
false_negatives = 0
#import torch.nn as nn
#import torch.nn.functional as F
# rust_dir = "/home/Hirra/coding_files/crossvit/results/wandb_vit_base_NWRD_epoch_50_lr_0.000000003_wd_0.001_batch_size_8_augmented_equlaized/rust"
# fn_dir = "/home/Hirra/coding_files/crossvit/results/wandb_vit_base_NWRD_epoch_50_lr_0.000000003_wd_0.001_batch_size_8_augmented_equlaized/fn"
# fp_dir = "/home/Hirra/coding_files/crossvit/results/wandb_vit_base_NWRD_epoch_50_lr_0.000000003_wd_0.001_batch_size_8_augmented_equlaized/fp"

rust_dir = "/home/Hirra/coding_files/crossvit/results/wandb_vit_base_updated_val_NWRD_epoch_50_lr_0.000000003_wd_0.001_batch_size_8_unaugmented_unequlaized/rust"
fn_dir = "/home/Hirra/coding_files/crossvit/results/wandb_vit_base_updated_val_NWRD_epoch_50_lr_0.000000003_wd_0.001_batch_size_8_unaugmented_unequlaized/fn"
fp_dir = "/home/Hirra/coding_files/crossvit/results/wandb_vit_base_updated_val_NWRD_epoch_50_lr_0.000000003_wd_0.001_batch_size_8_unaugmented_unequlaized/fp"

img_paths = test_ds.images
count=0
#softmax = nn.SoftMax()
model.eval
loop = tqdm(enumerate(test_loader), total=len(test_loader))
with torch.no_grad():
    for batch_idx, (images, labels) in loop:
        inputs = processor(images=images, return_tensors="pt", do_rescale=False).to(device)
        #print(inputs)
        labels = labels.to(device)

        outputs = model(**inputs)
        logits = outputs.logits
        #logits = F.softmax(logits)
        prediction = logits.argmax(axis=1)
        # print(logits)
        # print(labels)
        # print(prediction)
        if (prediction==1):
            image_path = os.path.join(rust_dir, img_paths[count].split('/')[-1])
            image = images.squeeze().cpu()
            #save_image(image,image_path)

        if ((prediction==1) and (labels==0)):
            image_path = os.path.join(fp_dir, img_paths[count].split('/')[-1])
            image = images.squeeze().cpu()
            #save_image(image,image_path)

        if ((prediction==0) and (labels==1)):
            image_path = os.path.join(fn_dir, img_paths[count].split('/')[-1])
            image = images.squeeze().cpu()
            #save_image(image,image_path)

        true_positives += torch.sum((prediction == 1) & (labels == 1)).item()
        false_positives += torch.sum((prediction == 1) & (labels == 0)).item()
        true_negatives += torch.sum((prediction == 0) & (labels == 0)).item()
        false_negatives += torch.sum((prediction == 0) & (labels == 1)).item()
        count+=1

# Calculate metrics
precision = true_positives / (true_positives + false_positives + 1e-10)
recall = true_positives / (true_positives + false_negatives + 1e-10)
F1 = 2 * (precision * recall) / (precision + recall)
accuracy = (true_positives + true_negatives) / (true_positives + false_positives + true_negatives + false_negatives + 1e-10)

# Print or use the metrics as needed
print(f"True Positives: {true_positives}")
print(f"False Positives: {false_positives}")
print(f"True Negatives: {true_negatives}")
print(f"False Negatives: {false_negatives}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1: {F1}")
print(f"Accuracy: {accuracy}")

100%|██████████| 3508/3508 [00:44<00:00, 79.62it/s]

True Positives: 551
False Positives: 232
True Negatives: 2619
False Negatives: 106
Precision: 0.7037037037036138
Recall: 0.8386605783864781
F1: 0.7652777777776716
Accuracy: 0.9036488027365763
